In [ ]:
import json
import requests
import simpleaudio
import time
import wave
import io

def play_wav_safely(wav_data):
    """
    WAVファイルを途切れることなく再生する関数
    """
    # 先頭に無音データを追加
    with wave.open(io.BytesIO(wav_data), 'rb') as wave_file:
        n_channels = wave_file.getnchannels()
        sampwidth = wave_file.getsampwidth()
        framerate = wave_file.getframerate()
        n_frames = wave_file.getnframes()
        wave_data = wave_file.readframes(n_frames)
    sec = 0.5
    padding_frames = int(sec * framerate)
    padding = b'\x00' * (padding_frames * n_channels * sampwidth)
    padded_data = padding + wave_data
    wave_obj = simpleaudio.WaveObject(padded_data, n_channels, sampwidth, framerate)
    play_obj = wave_obj.play()
    return play_obj

def generate_wav(text, speaker=1):
    start_time = time.time()
    host = 'localhost'
    port = 10101
    params = (
        ('text', text),
        ('speaker', speaker),
    )
    response1 = requests.post(
        f'http://{host}:{port}/audio_query',
        params=params
    )
    if response1.status_code != 200:
        print('get audio_query failed')
        print(response1)
        return
    headers = {
        'Content-Type': 'application/json',
        'accept': 'audio/wav'}
    response2 = requests.post(
        f'http://{host}:{port}/synthesis',
        headers=headers,
        params=params,
        data=json.dumps(response1.json())
    )
    if response2.status_code != 200:
        print('get synthesis failed')
        print(response2)
        return
    print(time.time() - start_time)
    play_obj = play_wav_safely(response2.content)
    play_obj.wait_done()
    # path = "test.wav"
    # with open(path, "wb") as f:
    #     f.write(response2.content)
    # print(f'generate_wav: {time.time() - start_time:.3f}')
    # wav_obj = simpleaudio.WaveObject.from_wave_file(path)
    # play_obj = wav_obj.play()
    # play_obj.wait_done()

def get_speaker_id():
    response = requests.get('http://localhost:10101/speakers')
    if response.status_code != 200:
        print('get speakers failed')
        print(response)
        return
    else:
        ids = []
        for speaker in response.json():
            print(speaker['name'])
            for style in speaker['styles']:
                print('    ' + style['name'] + ' ' + str(style['id']))
                ids.append(style['id'])
        return ids

if __name__ == '__main__':
    text = '貴方には一生縁のない話よ。興味を持たなくていいわ。'
    # ids = get_speaker_id()
    generate_wav(text, 888753765) # ids[5])

4.748401165008545
generate_wav: 4.748


In [6]:
nums = [1, 2, 3, 4, 5]
print(nums)
del nums[2:4]
print(nums)

[1, 2, 3, 4, 5]
[1, 2, 5]
